In [26]:
import random
from icecream import ic
import numpy as np
from tqdm import tqdm


In [ ]:
"""Create a random permutation of cities [0..n-1]."""
def random_solution(n):
    perm = list(range(n))
    random.shuffle(perm)
    return perm


"""Compute the total distance of the tour given by 'solution'."""
def route_length(solution, data):
    n = len(solution)
    total = 0.0
    #if a-b-c-d-a ,range(n); else if a-b-c-d , range(n-1)
    #b = solution[(i + 1) % n]; b = solution[i + 1]
    for i in range(n - 1):
        a = solution[i]
        b = solution[i + 1]
        total += float(data[a][b])
    return total

"""Sort solutions by tour length ascending (shorter is better)."""
def fitness_order(solutions, data):
    scored = [(s, route_length(s, data)) for s in solutions]
    scored.sort(key=lambda x: x[1])
    return [s for s, _ in scored]

"""Using Standard OX crossover to create a child solution from two parents."""
def ox_crossover(parent1, parent2):
    n = len(parent1)
    a, b = sorted(random.sample(range(n), 2))

    child = [None] * n
    child[a:b+1] = parent1[a:b+1]

    used = set(child[a:b+1])
    write = (b + 1) % n
    for g in parent2:
        if g not in used:
            child[write] = g
            write = (write + 1) % n

    return child

"""Perform swap mutation on perm solution"""
def swap(solution):
    n = len(solution)
    a, b = random.sample(range(n), 2)
    sol = solution[:]
    sol[a], sol[b] = sol[b], sol[a]
    return sol

"""Fitness-proportionate selection (shorter length -> higher weight)."""
"""For example, [10, 12, 13], max_len = 13, scores = [3, 1, 0],so probability of selecting each is [3/4, 1/4, 0]."""
def roulette(population, data):
    lengths = [route_length(s, data) for s in population]
    mx, mn = max(lengths), min(lengths)

    if mx == mn:
        # All have the same length, select randomly
        return random.choice(population)

    scores = [mx - L for L in lengths]  # > 0
    total = sum(scores)
    r, acc = random.random() , 0.0
    for s, w in zip(population, scores):
        acc += w
        if acc >= r:
            return s
    return population[-1]

In [28]:

"""Run GA for a single problem with OX crossover and swap mutation."""
def solve(data, populationSize, maxSteps, mutationProbability=0.2):
    n = data.shape[0]
    population = [random_solution(n) for _ in range(populationSize)]

    population = [random_solution(n) for _ in range(populationSize)]
    lengths = [route_length(s, data) for s in population]

    best_distance = min(lengths)
    best_solution = population[lengths.index(best_distance)][:]

    history = [best_distance]
    patience = 200        
    restart_frac = 0.30   
    no_improve = 0

    for _ in tqdm(range(maxSteps)):
        events = max(1, populationSize // 6)
        for _e in range(events):

            p1 = roulette(population, data)
            p2 = roulette(population, data)

            child = ox_crossover(p1, p2)
            if random.random() < mutationProbability:
                child = swap(child)
            Lc = route_length(child, data)

            worst_idx = max(range(populationSize), key=lambda i: lengths[i])
            if Lc < lengths[worst_idx]:
                population[worst_idx] = child
                lengths[worst_idx] = Lc

        # update best
        curr_idx = min(range(populationSize), key=lambda i: lengths[i])
        curr_best = lengths[curr_idx]
        if curr_best < best_distance:
            best_distance = curr_best
            best_solution = population[curr_idx][:]
            no_improve = 0
        else:
            no_improve += 1
        history.append(best_distance)

        # restart if no improvement and keep elites
        if no_improve >= patience:
            inject = max(1, int(populationSize * restart_frac))
            worst_indices = sorted(range(populationSize), key=lambda i: lengths[i], reverse=True)[:inject]
            for idx in worst_indices:
                population[idx] = random_solution(n)
                lengths[idx] = route_length(population[idx], data)
            curr_idx = min(range(populationSize), key=lambda i: lengths[i])
            best_distance = lengths[curr_idx]
            best_solution = population[curr_idx][:]
            no_improve = 0
            history.append(best_distance)

    return best_distance, best_solution, history

In [29]:
# Check data directory and load .npy files
DATA_DIR = './lab2'

import os
if not os.path.isdir(DATA_DIR):
    raise SystemExit(f"Data directory not found: {DATA_DIR}")

files = [f for f in os.listdir(DATA_DIR) if f.endswith('.npy')]
files.sort()
if not files:
    raise SystemExit("No .npy files found under ./lab2")

In [30]:
DEFAULT_POP = 10 
DEFAULT_STEPS = 10000 
DEFAULT_MUT = 0.2

all_results = []

In [ ]:
for fname in files:
    path = os.path.join(DATA_DIR, fname)
    data = np.load(path, allow_pickle=False)
    n = data.shape[0]

    pop = DEFAULT_POP
    steps = DEFAULT_STEPS
    mut = DEFAULT_MUT

    ic(f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}")
    bestDistance, bestSolution, history = solve(
        data=data,
        populationSize=pop,
        maxSteps=steps,
        mutationProbability=mut,
    )

    print("\n== Result ==")
    print(f"file: {fname}")
    print(f"bestDistance: {bestDistance}")
    print(f"bestSolution: {bestSolution}")
    all_results.append((fname, bestDistance, bestSolution))

print("\n=== SUMMARY (FULL) ===")
for name, dist_val, sol in all_results:
    print(f"file: {name}")
    print(f"bestDistance: {dist_val}")
    print(f"bestSolution: {sol}")
    print("-")

ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_10.npy -> N=10, pop=10, steps=10000, mut=0.2'
100%|██████████| 10000/10000 [00:00<00:00, 43352.34it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_100.npy -> N=100, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_g_10.npy
bestDistance: 1312.3976896345425
bestSolution: [2, 3, 1, 6, 4, 5, 9, 7, 0, 8]


100%|██████████| 10000/10000 [00:02<00:00, 4956.06it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_1000.npy -> N=1000, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_g_100.npy
bestDistance: 9545.522855583496
bestSolution: [47, 63, 71, 67, 0, 33, 54, 68, 81, 69, 64, 42, 72, 18, 56, 48, 14, 57, 31, 70, 2, 21, 10, 41, 84, 60, 16, 65, 62, 61, 23, 9, 59, 82, 39, 96, 76, 98, 26, 80, 19, 40, 25, 13, 95, 38, 45, 53, 79, 66, 3, 44, 17, 58, 28, 51, 85, 97, 30, 32, 29, 35, 90, 24, 15, 78, 83, 92, 87, 88, 4, 22, 1, 89, 75, 6, 91, 12, 43, 93, 34, 86, 55, 8, 77, 50, 52, 73, 94, 11, 74, 27, 20, 49, 7, 46, 36, 5, 37, 99]


100%|██████████| 10000/10000 [00:23<00:00, 425.47it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_g_20.npy -> N=20, pop=10, steps=10000, mut=0.2'


...

100%|██████████| 10000/10000 [00:04<00:00, 2471.62it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_r1_50.npy -> N=50, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_r1_200.npy
bestDistance: 4236.959279305829
bestSolution: [150, 60, 124, 9, 19, 4, 199, 102, 12, 65, 49, 23, 146, 5, 100, 86, 52, 139, 160, 180, 70, 111, 181, 96, 61, 194, 185, 192, 82, 21, 154, 144, 191, 1, 81, 112, 20, 153, 75, 183, 156, 71, 118, 62, 92, 117, 158, 6, 132, 104, 152, 120, 31, 26, 80, 159, 44, 11, 115, 113, 33, 107, 14, 184, 182, 87, 18, 88, 40, 91, 76, 148, 157, 119, 155, 43, 121, 36, 77, 103, 74, 195, 130, 83, 167, 137, 27, 163, 188, 177, 164, 94, 25, 56, 135, 145, 175, 106, 68, 171, 178, 168, 35, 0, 165, 15, 69, 140, 59, 134, 179, 97, 42, 95, 186, 79, 99, 196, 136, 122, 141, 190, 133, 176, 37, 98, 142, 149, 24, 126, 127, 63, 173, 39, 53, 110, 13, 32, 3, 84, 138, 85, 34, 143, 189, 50, 30, 172, 108, 128, 38, 162, 170, 151, 73, 48, 28, 54, 105, 29, 64, 131, 66, 90, 55, 101, 109, 93, 161, 89, 7, 114, 2, 125, 123, 147, 51, 58, 116, 193, 57, 166, 10, 129, 72, 22, 197, 174, 17, 45, 8, 78, 47, 16, 198, 67, 41, 187, 169, 46]


100%|██████████| 10000/10000 [00:01<00:00, 9645.94it/s]
ic| f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}": 'Running problem_r1_500.npy -> N=500, pop=10, steps=10000, mut=0.2'



== Result ==
file: problem_r1_50.npy
bestDistance: 926.3947156709879
bestSolution: [45, 6, 30, 1, 20, 21, 26, 7, 46, 49, 14, 24, 2, 11, 44, 33, 39, 12, 13, 0, 23, 37, 10, 34, 3, 27, 15, 42, 31, 19, 47, 38, 32, 43, 36, 22, 40, 29, 5, 17, 16, 8, 9, 35, 41, 48, 28, 18, 25, 4]


 51%|█████     | 5065/10000 [00:06<00:05, 822.80it/s]